# Defining an Architecture

In [0]:
from skimage import io
from skimage import transform

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

!wget https://upload.wikimedia.org/wikipedia/en/thumb/7/7d/Lenna_%28test_image%29.png/220px-Lenna_%28test_image%29.png

## Extending a Module

A customized neural network is implemented as a class that extends the superclass $nn.Module$. The most common methods are $\_\_init\_\_()$ and $forward()$.

In [0]:
# Customized Network.
class CustomNetwork(nn.Module):
    
    def __init__(self, in_channels, num_classes=2):

        super(CustomNetwork, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(in_channels, 4, 3, 1, 1),
            nn.BatchNorm2d(4),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2, stride=2), # 256 -> 128

            nn.Conv2d(4, 8, 3, 1, 1),
            nn.MaxPool2d(2, stride=2),
            nn.BatchNorm2d(8),
            nn.ReLU(inplace=True), # 128 -> 64

            nn.Conv2d(8, 16, 3, 1, 1),
            nn.MaxPool2d(2, stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True), # 64 -> 32

            nn.Conv2d(16, 32, 3, 1, 1),
            nn.MaxPool2d(2, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True), # 32 -> 16

            nn.Conv2d(32, 32, 1, 1, 0),
            nn.MaxPool2d(2, stride=2),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True), # 16 -> 8

            nn.Conv2d(32, 16, 1, 1, 0),
            nn.MaxPool2d(2, stride=2),
            nn.BatchNorm2d(16),
            nn.ReLU(inplace=True), # 8 -> 4
        )

        self.classifier = nn.Sequential(
            nn.Linear(16 * 4 * 4, 256),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(256, 128),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(128, num_classes),
        )
        
        self.initialize_weights()
    
    def initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
            elif isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, 0, 0.01)
                nn.init.constant_(m.bias, 0)
                
    def forward(self, x):
        
        # Feature extraction.
        x = self.features(x)
        #print('feats: ', x.size())
        
        # Changing tensor shape.
        x = x.view(x.size(0), -1)
        #print('new view: ', x.size())
        
        # Classification.
        x = self.classifier(x)
        #print('class: ', x.size())
        
        return x

## Loading image.

Image load can be done using any image library. Pytorch has a direct link with the $PIL$ library.

In [0]:
# Loading image, resizing to 256x256 and normalizing.
img = io.imread('220px-Lenna_(test_image).png')
io.imshow(img)
print(img.shape)
img = transform.resize(img, (256, 256)).astype(np.float32)
if len(img.shape) > 2:
    img = img[:,:,0]
img = (img - img.mean()) / img.std()

print(img.shape)

## From ndarray to tensor

If one choses an image lib that loads images as ndarrays (such as $scikit-image$), transforming to tensor prior to feeding the image to the network is necessary.

In [0]:
# Transforming to tensor.
tensor = torch.from_numpy(img)
tensor = tensor.unsqueeze(0)
tensor = tensor.unsqueeze(0)
# tensor = tensor.cuda() # For GPU casting.

print(tensor.size())
print(tensor)

## Forwarding tensor

Forwarding tensor through network.

In [0]:
# Forwarding image.
in_channels = 1
num_classes = 2

net = CustomNetwork(in_channels, num_classes)
# net = CustomNetwork(in_channels, num_classes).cuda() # For GPU casting.

output = net(tensor)
prob = F.softmax(output)

print('output: ', output)
print('class probabilities: ', prob)